[![alt text](https://avatars1.githubusercontent.com/u/59831504?s=400&v=4 "MtheEPIC User Icon")](https://github.com/MtheEPIC/KaggleProj)

# Import libs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.axes_style("whitegrid")

import glob

from sklearn.decomposition import PCA

from sklearn.preprocessing import MinMaxScaler, binarize
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, plot_roc_curve, confusion_matrix, classification_report, precision_recall_curve, average_precision_score  
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, cross_val_predict

from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC

In [2]:
def mod_df(df, df2=None):
    df.rename(str.strip, axis='columns', inplace=True)
#     for i in range(len(df.columns)):
#         df.iloc[:, i] = df.iloc[:, i].strip()
    df = df[df['Time']>=df.iloc[0, 0]+7]
    
    start_time = df.iloc[0, 0]
    end_time = df.iloc[len(df)-1, 0]
    total_time = (end_time - start_time)
    start_frame = df.iloc[0, 1]
    end_frame = df.iloc[len(df)-1, 1]
    total_frame = (end_frame - start_frame)
#     if df2 is None:
# #         print("here")
# #         print('')
# #         jump = total_frame//total_time
#         df['tmp'] = np.arange(df.shape[0])
#         df_cut = df[df['tmp'].isin(np.linspace(0, df.shape[0], df.shape[0]//24))]
#         df_cut.drop('tmp', axis=1)
#     else:
#         df_cut = df
#         size = df.shape[0]
#         print('size ', df2.shape[0])
#         jump = total_frame//size
#         df_cut = df[df['Frame ID'].isin(np.linspace(start_frame, end_frame, size))]
    
    df['tmp'] = np.arange(df.shape[0])
    df_cut = df[df['tmp'].isin(np.linspace(0, df.shape[0], df.shape[0]//24))]
    df_cut.drop('tmp', axis=1)
    
    
#     jump = total_frame//100
#     print(start_time, ' ', end_time, ' ', jump, ' ', start_frame, ' ', end_frame)
    
#     print(df_cut.head())
    return df_cut

In [3]:
# tmp = mod_df(train_spontan)
# # train_spontan.iloc[0, 0]
# tmp.info()
# tmp.head(22)

In [4]:
train_sync = pd.read_csv('data/Training/Evyatar Cohen/Evyatar636771050902934708Sync.csv')
# test_df = pd.read_csv('data/fashion-mnist_test.csv')
train_sync.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,153.5306,107537,right,2,77.94024,254.7147,-7.674313,-0.480022,-59.95172,127.34810,...,-0.332276,101.0865,198.4052,27.67238,248.6154,17.15779,115.17400,0.0,0.0,0.0
1,153.5306,107537,left,2,-103.68820,221.1642,3.092919,-2.858919,-180.74000,55.11667,...,0.866406,-133.3421,158.3205,14.84969,-268.8730,-49.14569,44.19444,0.0,0.0,0.0
2,153.5476,107539,right,2,78.28185,253.5882,-5.193746,24.038670,-51.37317,143.18080,...,-0.332286,101.4179,197.1837,29.99536,250.0403,15.59908,114.90930,0.0,0.0,0.0
3,153.5476,107539,left,2,-103.54540,218.4241,4.356668,10.185880,-126.31100,68.19558,...,0.884464,-133.7134,155.7733,15.84225,-269.5829,-51.50067,44.97860,0.0,0.0,0.0
4,153.5644,107541,right,2,78.85633,252.8531,-2.370064,43.635580,-31.83131,169.02340,...,-0.325577,101.7099,196.2052,32.58978,251.4731,14.51754,115.24890,0.0,0.0,0.0


## data mod

In [5]:
# train_spontan.iloc[0, 0]

In [6]:
# train_spontan['Time']

In [7]:
# from os import listdir
# from os.path import isfile, join

# mypath = os.path.dirname(os.path.realpath(__file__))
# print(mypath)
# # onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [11]:
cnt = 0



r_hand_path = 'data\\HandRight.csv'
path = 'data\\Training\\*\\*'
train_df = pd.DataFrame()
r_hand_df = pd.read_csv(r_hand_path)
r_hand_df.dropna(inplace=True)
r_hand_df['value'] = 0
print(r_hand_df.shape)

for name in glob.glob(path):
    cnt = cnt +1
    print(name)
    df = pd.read_csv(name)
    print('origen', df.shape[0])
    
    df.dropna(inplace=True)
    if 'Alone' in name[-11:]: #add r hand
        df = mod_df(df, r_hand_df)
        df['value'] = 0
    else:
        df  = mod_df(df)
        if 'Spontan' in name[-11:]:
            df['value'] = 1
        elif 'Sync' in name[-11:]:
            df['value'] = 2
        else:
            print('ERORR')
    print('after', df.shape)
#     print(df.head())
cnt

(4428, 23)
data\Training\Evyatar Cohen\Evyatar636771050011691836Spontan.csv
origen 9054
after (341, 24)
data\Training\Evyatar Cohen\Evyatar636771050902934708Sync.csv
origen 9054
after (341, 24)
data\Training\Evyatar Cohen\Evyatar636771051823980826Alone.csv
origen 4476
after (167, 24)
data\Training\Evyatar Cohen\Evyatar636771052727603804Spontan.csv
origen 9054
after (341, 24)
data\Training\Evyatar Cohen\Evyatar636771053639929594Sync.csv
origen 9052
after (340, 24)
data\Training\Evyatar Cohen\Evyatar636771054555711409Alone.csv


C:\Users\mthee\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


origen 4526
after (169, 24)
data\Training\Nofar Social_Nuero\Nofar636759792183061005Spontan.csv
origen 9054
after (341, 24)
data\Training\Nofar Social_Nuero\Nofar636759793193137655Alone.csv
origen 4496
after (168, 24)
data\Training\Nofar Social_Nuero\Nofar636759794200223541Sync.csv
origen 9052
after (340, 24)
data\Training\Nofar Social_Nuero\Nofar636759795182793299Spontan.csv
origen 8998
after (338, 24)
data\Training\Nofar Social_Nuero\Nofar636759796290435160Alone.csv
origen 4575
after (172, 24)
data\Training\Nofar Social_Nuero\Nofar636759797397919664Sync.csv
origen 9054
after (341, 24)
data\Training\Oriya Social_Nuero\Oriya636759799659227614Spontan.csv
origen 9011
after (339, 24)
data\Training\Oriya Social_Nuero\Oriya636759800632839612Alone.csv
origen 4367
after (164, 24)
data\Training\Oriya Social_Nuero\Oriya636759801991504932Alone.csv
origen 4500
after (168, 24)
data\Training\Oriya Social_Nuero\Oriya636759803164759865Sync.csv
origen 9054
after (341, 24)
data\Training\Oriya Social_Nu

47

In [9]:
tmp = tmp.set_index(np.arange(0, tmp.shape[0]*2, 2))
tmp

NameError: name 'tmp' is not defined

In [ ]:
# r_hand_df = pd.read_csv(r_hand_path)
# rrrr = r_hand_df
r_hand_df

In [ ]:
r_hand_df = r_hand_df.set_index(np.arange(1, r_hand_df.shape[0]*2, 2))
r_hand_df

In [ ]:
r_hand_df.join(tmp, how='outer')
r_hand_df